## Please input your directory for the top level folder
folder name : SUBMISSION MODEL

In [1]:
dir_ = 'C:/Users/jekim/OneDrive/바탕 화면/m5-forecasting-accuracy/' # input only here

#### setting other directory

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'
submission_dir = dir_+'6. submissions/'

In [3]:
####################################################################################
##################### 1-3. recursive model by store & dept #########################
####################################################################################

In [4]:
ver, KKK = 'priv', 0

In [5]:
STORES = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
DEPTS = ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1', 'FOODS_2', 'FOODS_3']
CLUSTERS = [0,1,2,3,4,5,6,7,8,9]

In [6]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool

warnings.filterwarnings('ignore')

In [7]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [8]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_cluster(cluster):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    df = df[df['d']>=START_TRAIN]
    
    df = df[df['tskm_10']==cluster]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3
    
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for cluster_id in CLUSTERS:
        temp_df = pd.read_pickle(processed_data_dir+'test_cluster10_'+str(cluster_id)+'.pkl')
        temp_df['cluster_id'] = cluster_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [9]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**8-1,
                    'min_data_in_leaf': 2**8-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': -1
                } 

In [10]:
########################### Vars
#################################################################################
VER = 1                          
SEED = 42                        
seed_everything(SEED)            
lgb_params['seed'] = SEED        
N_CORES = psutil.cpu_count()     


#LIMITS and const
TARGET      = 'sales'            
START_TRAIN = 700                
END_TRAIN   = 1941 - 28*KKK      
P_HORIZON   = 28                 
USE_AUX     = False             

remove_features = ['id', 'dept_id', 'cat_id', 'store_id', 'state_id',
                   'tskm_7', 'tskm_5', 'tskm_30', 'tskm_50',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_item_id_store_id_mean','enc_item_id_store_id_std'] 

ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'grid_part_1.pkl'
PRICE    = processed_data_dir+'grid_part_2.pkl'
CALENDAR = processed_data_dir+'grid_part_3.pkl'
LAGS     = processed_data_dir+'lags_df_28.pkl'
MEAN_ENC = processed_data_dir+'mean_encoding_df.pkl'


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [11]:
_, MODEL_FEATURES = get_data_by_cluster(CLUSTERS[-1])
del _; gc.collect()

0

In [12]:
########################### Predict
#################################################################################

all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

main_time = time.time()

for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    grid_df = base_test.copy()
    
    # slow for loop version
    temp = []
    for a in ROLS_SPLIT:
        temp.append(make_lag_roll(a))
    temp = pd.concat(temp, axis=1)
    grid_df = pd.concat([grid_df, temp], axis=1)
    del temp; gc.collect()
    ###
    
    # fast multiprocessing version
#     grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
    ###
    
    for cluster_id in CLUSTERS:
        
        model_path = model_dir+'lgb_model_cluster10_'+str(cluster_id)+'_v'+str(VER)+'.bin'
        if USE_AUX:
            model_path = AUX_MODELS + model_path

        estimator = pickle.load(open(model_path, 'rb'))

        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        cluster_mask = base_test['cluster_id']==cluster_id

        mask = (day_mask)&(cluster_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

Predict | Day: 1
##########  4.17 min round |  4.17 min total |  40010.16 day sales |
Predict | Day: 2
##########  3.98 min round |  8.15 min total |  37281.01 day sales |
Predict | Day: 3
##########  3.71 min round |  11.86 min total |  36976.32 day sales |
Predict | Day: 4
##########  3.92 min round |  15.79 min total |  37284.36 day sales |
Predict | Day: 5
##########  3.71 min round |  19.50 min total |  42429.32 day sales |
Predict | Day: 6
##########  3.89 min round |  23.39 min total |  50459.54 day sales |
Predict | Day: 7
##########  3.86 min round |  27.25 min total |  51396.71 day sales |
Predict | Day: 8
##########  3.73 min round |  30.98 min total |  45905.86 day sales |
Predict | Day: 9
##########  3.88 min round |  34.86 min total |  39340.13 day sales |
Predict | Day: 10
##########  3.83 min round |  38.69 min total |  44144.61 day sales |
Predict | Day: 11
##########  3.76 min round |  42.45 min total |  44945.57 day sales |
Predict | Day: 12
##########  3.86 min roun

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_2_001_CA_1_evaluation,0.151890,0.153159,0.170719,0.163684,0.154708,0.153024,0.162855,0.157862,0.150981,...,0.165108,0.138427,0.163167,0.129707,0.129276,0.145366,0.136718,0.142118,0.148751,0.120125
1,HOBBIES_2_002_CA_1_evaluation,0.179065,0.171663,0.175491,0.215790,0.205928,0.200002,0.192711,0.216036,0.182361,...,0.142019,0.138986,0.155118,0.175742,0.150152,0.161958,0.182883,0.158953,0.180786,0.186196
2,HOBBIES_2_003_CA_1_evaluation,0.256544,0.275141,0.223164,0.262014,0.285835,0.347447,0.407258,0.351635,0.269963,...,0.377654,0.462655,0.467815,0.285932,0.367465,0.310105,0.312765,0.348017,0.385164,0.326125
3,HOBBIES_2_004_CA_1_evaluation,0.267795,0.235994,0.220327,0.200279,0.217907,0.365538,0.347698,0.309666,0.202800,...,0.235895,0.333507,0.327528,0.215966,0.190852,0.206354,0.204125,0.229004,0.337773,0.270497
4,HOBBIES_2_005_CA_1_evaluation,0.150335,0.177847,0.179095,0.210062,0.186996,0.220952,0.180146,0.181431,0.185185,...,0.207700,0.221585,0.227435,0.167161,0.194182,0.200280,0.181875,0.193336,0.244475,0.238558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_1_215_WI_2_evaluation,1.023224,0.865696,1.029047,0.948048,0.974396,1.256507,1.252827,1.105913,1.049686,...,1.080299,1.326417,1.361569,0.974662,1.031241,1.173638,1.125283,1.051830,1.193993,1.213580
30486,FOODS_1_216_WI_2_evaluation,0.748372,0.720366,0.732056,0.715039,0.908937,0.972567,0.956063,0.760630,0.737582,...,0.827755,0.917397,1.029784,0.798495,0.748967,0.790388,0.684090,0.866819,0.877255,0.965761
30487,FOODS_1_217_WI_2_evaluation,1.870141,1.809590,1.848161,1.953223,2.079475,2.210190,1.988771,2.072662,1.848150,...,2.216131,2.483877,2.201126,2.134844,1.860835,2.111912,1.888020,2.279217,2.489347,2.054295
30488,FOODS_1_218_WI_2_evaluation,11.781188,11.046625,11.326746,11.685886,14.897804,20.634109,16.842920,13.036892,13.573682,...,18.946629,24.012286,19.470446,16.310815,16.557525,16.516895,16.464363,20.561653,24.993195,19.137149


In [13]:
########################### Export
#################################################################################
submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv(submission_dir+'before_ensemble/submission_kaggle_recursive_store_dept.csv', index=False)